In [3]:
import pandas as pd

In [4]:
te_projections = pd.read_csv("/content/drive/MyDrive/FantasyFootbalGIGASTATS/Raw/8-26-2023/FantasyPros_Fantasy_Football_Projections_TE (6).csv")
wr_projections = pd.read_csv("/content/drive/MyDrive/FantasyFootbalGIGASTATS/Raw/8-26-2023/FantasyPros_Fantasy_Football_Projections_WR (7).csv")
rb_projections = pd.read_csv("/content/drive/MyDrive/FantasyFootbalGIGASTATS/Raw/8-26-2023/FantasyPros_Fantasy_Football_Projections_RB (7).csv")
qb_projections = pd.read_csv("/content/drive/MyDrive/FantasyFootbalGIGASTATS/Raw/8-26-2023/FantasyPros_Fantasy_Football_Projections_QB (6).csv")


In [5]:
te_projections.head(10)

,Player,Team,REC,YDS,TDS,FL,FPTS
0,,NaN,NaN,NaN,NaN,NaN,NaN
1,Travis Kelce,KC,105.5,"1,236.8",9.9,0.6,234.5
2,NaN,high,119.8,"1,361.5",12.6,0.0,360.6
3,NaN,low,94.0,"1,144.0",7.9,1.0,219.8
4,Mark Andrews,BAL,74.8,884.1,6.4,0.2,165.5
5,NaN,high,85.5,949.2,7.9,0.0,235.4
6,NaN,low,66.0,844.0,5.1,0.6,148.4
7,T.J. Hockenson,MIN,80.9,833.4,5.7,0.5,157.0
8,NaN,high,88.6,877.0,7.4,0.0,232.8
9,NaN,low,75.2,792.2,4.9,1.0,140.5


In [6]:
def transform_dataframe(df):
    # Create empty lists to store processed data
    names, teams, fpts, high_values, low_values = [], [], [], [], []

    # Iterate through the DataFrame in steps of 3
    for i in range(1,len(df)-2, 3):
        # Extract values
        names.append(df.iloc[i]['Player'])
        teams.append(df.iloc[i]['Team'])
        fpts.append(df.iloc[i]['FPTS'])
        high_values.append(df.iloc[i+1]['FPTS'])
        low_values.append(df.iloc[i+2]['FPTS'])

    # Create a new DataFrame using the extracted values
    new_df = pd.DataFrame({
        'Player': names,
        'Team': teams,
        'FPTS': fpts,
        'High_FPTS': high_values,
        'Low_FPTS': low_values
    })

    return new_df

In [7]:
new_te = transform_dataframe(te_projections)
new_wr = transform_dataframe(wr_projections)
new_rb = transform_dataframe(rb_projections)
new_qb = transform_dataframe(qb_projections)
new_te.head(10)

,Player,Team,FPTS,High_FPTS,Low_FPTS
0,Travis Kelce,KC,234.5,360.6,219.8
1,Mark Andrews,BAL,165.5,235.4,148.4
2,T.J. Hockenson,MIN,157.0,232.8,140.5
3,Darren Waller,NYG,145.4,231.6,130.7
4,George Kittle,SF,140.7,208.1,132.6
5,Kyle Pitts,ATL,132.1,193.8,122.6
6,Dallas Goedert,PHI,129.6,223.6,117.6
7,Evan Engram,JAC,118.8,179.5,113.3
8,Pat Freiermuth,PIT,117.6,164.7,115.0
9,Dalton Schultz,HOU,109.6,177.3,102.4


In [8]:
#some redditor came up with this lol idk how baselines work
#Values are wacky af but at least someone put thought into them
#Other baselines are as follows:
#FootBallAbsurdity = {'QB': 25, 'RB': 33, 'WR': 41, 'TE': 12}
#SleeperADPBased = {'QB':34, 'RB': 48, 'WR': '60', 'TE': 19}

par_qb = new_qb.iloc[25]['FPTS']  # 15th ranked QB
par_rb = new_rb.iloc[33]['FPTS']  # 37th ranked RB
par_wr = new_wr.iloc[41]['FPTS']  # 44th ranked WR
par_te = new_te.iloc[14]['FPTS']  # 17th ranked TE

par_qb, par_rb, par_wr, par_te

(223.2, 138.9, 136.6, 99.0)

In [9]:
def compute_features(df, par):
    # VAL calculation
    par = par/16
    df['VAL'] = (df['FPTS']/16) - par

    # F (Low projection VAL) and C (High projection VAL) calculations
    df['F'] = (df['Low_FPTS'] / 16) - par
    df['C'] = (df['High_FPTS'] / 16) - par

    return df

new_te = compute_features(new_te,par_te)
new_rb = compute_features(new_rb,par_rb)
new_qb = compute_features(new_qb,par_qb)
new_wr = compute_features(new_wr,par_wr)


In [10]:
def compute_ps(df):
    df_sorted = df.sort_values(by='VAL', ascending=False).reset_index(drop=True)
    df_sorted['Cumulative_VAL'] = df_sorted['VAL'].cumsum()
    total_positive_val = df_sorted[df_sorted['VAL'] > 0]['VAL'].sum()
    df_sorted['PS'] = df_sorted['Cumulative_VAL'].apply(lambda x: ((total_positive_val - x) / total_positive_val)*100).round(0)
    df_sorted.loc[df_sorted['VAL'] < 0, 'PS'] = 0
    return df_sorted

new_te = compute_ps(new_te)
new_rb = compute_ps(new_rb)
new_wr = compute_ps(new_wr)
new_qb = compute_ps(new_qb)

In [11]:
new_rb.head(10)
new_qb.head(10)

,Player,Team,FPTS,High_FPTS,Low_FPTS,VAL,F,C,Cumulative_VAL,PS
0,Patrick Mahomes II,KC,384.0,450.5,366.5,10.05000,8.95625,14.20625,10.05000,91.0
1,Josh Allen,BUF,380.6,446.5,357.5,9.83750,8.39375,13.95625,19.88750,83.0
2,Jalen Hurts,PHI,376.5,424.0,337.5,9.58125,7.14375,12.55000,29.46875,74.0
3,Lamar Jackson,BAL,351.2,396.1,327.7,8.00000,6.53125,10.80625,37.46875,67.0
4,Joe Burrow,CIN,346.1,400.1,331.4,7.68125,6.76250,11.05625,45.15000,60.0
5,Justin Fields,CHI,326.9,375.8,308.9,6.48125,5.35625,9.53750,51.63125,55.0
6,Justin Herbert,LAC,321.6,383.1,296.1,6.15000,4.55625,9.99375,57.78125,49.0
7,Deshaun Watson,CLE,311.5,338.0,297.3,5.51875,4.63125,7.17500,63.30000,44.0
8,Dak Prescott,DAL,301.1,342.9,281.8,4.86875,3.66250,7.48125,68.16875,40.0
9,Trevor Lawrence,JAC,300.9,348.3,282.3,4.85625,3.69375,7.81875,73.02500,36.0


In [12]:
cols_to_keep = ['Player', 'Team', 'FPTS','High_FPTS', 'Low_FPTS', 'VAL', 'F', 'C', 'PS']
combined_df = pd.concat([
    new_qb[cols_to_keep].assign(Position='QB'),
    new_rb[cols_to_keep].assign(Position='RB'),
    new_wr[cols_to_keep].assign(Position='WR'),
    new_te[cols_to_keep].assign(Position='TE')
], ignore_index=True)

# Save the combined dataframe to a CSV file named "DanSheets.csv"
file_path = "/content/drive/MyDrive/FantasyFootbalGIGASTATS/Clean/DanSheetscustomAug26.csv"
combined_df.to_csv(file_path, index=False)